In [ ]:
# Import the necessary modules
from pyspark.sql.functions import *
from ultis import *
from datetime import datetime
from decouple import config

spark = create_spark()

In [ ]:
spark.sql('''
SELECT
    A.DATE,
    COALESCE(B.PRODUCT, 0) AS PRODUCT,
    COALESCE(B.FACILITY, 0) AS FACILITY,
    COALESCE(B.SERVICE, 0) AS SERVICE,
    FROM_UTC_TIMESTAMP(NOW(), 'GMT+7') AS LAST_EXECUTED_TIME
FROM delta.`s3a://silver/CALENDAR_DIM` A
LEFT JOIN ((
SELECT
    TO_DATE(FROM_UTC_TIMESTAMP(A.CREATED_AT, 'GMT+7')) AS DATE,
    COUNT(*) AS TICKET_AMOUNT,
    B.CASE_CATEGORY_1_NAME
FROM delta.`s3a://silver/USER_RATING_FACT` A
LEFT JOIN delta.`s3a://silver/CASE_CATEGORY_DIM` B ON A.CASE_CATEGORY_ID = B.CASE_CATEGORY_ID
GROUP BY TO_DATE(FROM_UTC_TIMESTAMP(A.CREATED_AT, 'GMT+7')), CASE_CATEGORY_1_NAME
)
PIVOT (
SUM(TICKET_AMOUNT) FOR CASE_CATEGORY_1_NAME IN (
'Product' PRODUCT, 'Facility' FACILITY, 'Service' SERVICE
)
)) B ON A.DATE = B.DATE
''').write.format('delta').mode('overwrite').save('s3a://gold/USER_RATING_CATEGORY')

In [ ]:
spark.stop()